In [ ]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:


def load_dataset():
  image_pickle_file_path = '/content/drive/MyDrive/project/images.pkl'
  label_pickle_file_path = '/content/drive/MyDrive/project/ label.pkl'

  with open(image_pickle_file_path, 'rb') as file:
    images = pickle.load(file)

  with open(label_pickle_file_path, 'rb') as file:
    labels = pickle.load(file)

  images = images.reshape(images.shape[0], -1)

  return images, labels

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Classify the datapoints with the Random Forest Classifier

def classify(datapoints, labels):
  test_size = 0.2
  X_train, X_test, y_train, y_test = train_test_split(datapoints, labels, test_size=test_size, random_state=42)

  clf = RandomForestClassifier(n_estimators=100, random_state=42)

  clf.fit(X_train, y_train)

  y_pred = clf.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
images, true_labels = load_dataset()
images[0]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import colorsys
def convertToHSV(image):

  pixels = np.array(image)
  rgb_image = pixels.reshape((499, 499,3))
  r, g, b = rgb_image[:, :, 0], rgb_image[:, :, 1], rgb_image[:, :, 2]
  vectorized_rgb_to_hsv = np.vectorize(colorsys.rgb_to_hsv)
  hsv_image = vectorized_rgb_to_hsv(r / 255.0, g / 255.0, b / 255.0)
  hsv_image = np.transpose(hsv_image, (1, 2, 0))
  return hsv_image


newimg=convertToHSV(images[33])
# original=np.array(images[33]).reshape((499, 499,3))
# for i in range(0,10):
#   print('rgb:',original[i][0])
#   print('hsv',newimg[i][0])
#   print("--------")
plt.imshow(newimg, cmap='hsv')
plt.axis('off')
plt.show()

In [ ]:

# def deleteGreen(image):
#     hsv_image = convertToHSV(image)
#     h, s, v = hsv_image[:, :, 0], hsv_image[:, :, 1], hsv_image[:, :, 2]
#     green_pixels = (h >= 0.3) & (h <= 0.5) #& (s >= 0.3)
#     hsv_image[green_pixels, :] = 0
#     return hsv_image

# newimg=deleteGreen(images[33])
# plt.imshow(newimg, cmap='hsv')

# plt.axis('off')
# plt.show()

In [ ]:
# img=images

# plt.subplot(1, 3, 1)
# plt.imshow(np.array(img).reshape((499, 499,3)))

# plt.subplot(1, 3, 2)
# newimg=convertToHSV(img)
# plt.imshow(newimg, cmap='hsv')

# plt.subplot(1, 3, 3)
# newimg=deleteGreen(img)
# plt.imshow(newimg, cmap='hsv')
# plt.axis('off')
# plt.show()

In [ ]:
import math

def convertToPolar(i,j):
    center=249
    dx = i - center
    dy = j - center
    distance = math.sqrt(dx**2 + dy**2)
    angle_rad = math.atan2(dy, dx)
    angle_degrees = math.degrees(angle_rad)
    return distance, angle_degrees



In [ ]:

import numpy as np
from sklearn.preprocessing import MinMaxScaler

def create(image):
    pixels={}
    pixels['info'] = []
    pixels['x']=[]
    pixels['y']=[]
    scaler = MinMaxScaler()

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            pixel = image[i, j]
            distance, angle = convertToPolar(i, j)
            normalized_pixel = [pixel[0], pixel[1], pixel[2] ,distance, angle]
            pixels['info'].append(normalized_pixel)
            pixels['x'].append(j)
            pixels['y'].append(500-i)

    pixels['info'] = np.array(pixels['info'])
    pixels['info'] = scaler.fit_transform(pixels['info'])*np.array([18,6,6,1,1])
    return pixels

# normalized_data = create(newimg)
# images[33][1][1]
# normalized_data['info'][:10]

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# wcss = []
# data = create(newimg)
# for n_clusters in range(1,7):
#     kmeans = KMeans(n_clusters=n_clusters, n_init=100)
#     kmeans.fit(data['info'])
#     wcss.append(kmeans.inertia_)
#     print(wcss)


# plt.plot(range(1, 7), wcss)
# plt.title('Elbow Method')
# plt.xlabel('Number of Clusters')
# plt.ylabel('WCSS')
# plt.show()


data = create(newimg)
kmeans = KMeans(n_clusters=4, n_init=10, random_state=42).fit(data['info'])
labels = kmeans.labels_

# pca = PCA(n_components=3)
# reduced_data = pca.fit_transform(data)

plt.scatter(data['x'], data['y'], c=labels, cmap='viridis')
plt.title('K-means Clustering')
plt.show()


In [ ]:
import cv2

#
# data = create(newimg)
def groupByLabel(data,labels):
  group={}
  for info,x,y,label in zip(data['info'],data['x'],data['y'],labels):
    if(label not in group.keys()):

      group[label]={}
      group[label]['x']=[]
      group[label]['y']=[]
      group[label]['info']=[]

    group[label]['info'].append(info)
    group[label]['x'].append(x)
    group[label]['y'].append(y)
  return group

def colorVariance(img,group):
  total_variance=0
  h_list=[]
  s_list=[]
  v_list=[]
  for x,y in zip(group['x'],group['y']):
    color= img[500-y][x]
    h_list.append(color[0])
    s_list.append(color[1])
    v_list.append(color[2])
  h_variance = np.var(h_list)
  s_variance = np.var(s_list)
  v_variance = np.var(v_list)
  # total_variance = h_variance  + s_variance + v_variance

  return h_variance , s_variance , v_variance

def colorAverage(img,group):
  total_variance=0
  h_list=[]
  s_list=[]
  v_list=[]
  for x,y in zip(group['x'],group['y']):
    color= img[500-y][x]
    h_list.append(color[0])
    s_list.append(color[1])
    v_list.append(color[2])
  h_variance = np.average(h_list)
  s_variance = np.average(s_list)
  v_variance = np.average(v_list)
  # total_variance = h_variance  + s_variance + v_variance

  return h_variance , s_variance , v_variance

In [ ]:
def hAverage(img,group):
  total_variance=0
  h_list=[]
  for x,y in zip(group['x'],group['y']):
    color= img[500-y][x]
    h_list.append(color[0])
  h_variance = np.average(h_list)
  h_min=np.amin(h_list)
  h_max=np.amax(h_list)
  return h_variance,h_min,h_max

In [ ]:
def pointVariance(group):
  # print(group['info'][0:3])
  three=[]
  four=[]
  for i in group['info']:
    three.append(i[3])
    four.append(i[4])
  # total_variance=np.var(three)

  return np.var(three),np.var(four)

def pointAverage(group):
  # print(group['info'][0:3])
  three=[]
  four=[]
  for i in group['info']:
    three.append(i[3])
    four.append(i[4])
  # total_variance=np.var(three)

  return np.average(three),np.average(four)


In [ ]:
#محیط
from shapely.geometry import Point, MultiPoint

def calculaterPerimeter(group):
  point_coordinates = [Point(x, y) for x,y in zip(group['x'],group['y']) ]#zip(test_coordinates['x'],test_coordinates['y'])
  multi_point = MultiPoint(point_coordinates)
  convex_hull = multi_point.convex_hull

  bounding_box = multi_point.bounds
  return convex_hull.length

In [ ]:

def circularity(group):
  primeter=calculaterPerimeter(group)
  area=len(group['x'])
  return 4*np.pi*area/(primeter**2)


In [ ]:
!pip install tqdm
from tqdm import tqdm


In [ ]:
def aspect_ratio(group):
  # for x,y in zip(group['x'],group['y']):
  ymax=np.amax(group['x'])
  ymin=np.amin(group['x'])
  xmax=np.amax(group['y'])
  xmin=np.amin(group['y'])
  ydif=ymax-ymin
  xdif=xmax-xmin
  return (ydif /xdif)




In [ ]:
scaler = MinMaxScaler()
def findFeatures(images):
  features=[]
  vectors=[]
  i=0
  for image in images:
    imgvectors=[]
    hsvImg=convertToHSV(image)#deleteGreen(image)#
    data = create(hsvImg)
    kmeans = KMeans(n_clusters=4, n_init=10, random_state=42).fit(data['info'])
    labels = kmeans.labels_
    centers=kmeans.cluster_centers_
    data['centers_x']=centers[:,3]
    data['centers_y']=centers[:,4]

    unique, counts = np.unique(labels, return_counts=True)
    groups=groupByLabel(data,labels)
    for g,c in zip(unique,counts):
      groups[g]['size']=c

      # groups[g]['center_x']=data['centers_x'][g]
      # groups[g]['center_y']=data['centers_y'][g]

    for v in groups.values():
        dist,ang=pointAverage(v)#pointVariance(v)
        # hcolor,scolor,vcolor=colorVariance(hsvImg,v)
        # h_avg,h_min,h_max=hAverage(hsvImg,v)
        h_avg,s_avg,v_avg=colorAverage(hsvImg,v)

        vector=[h_avg,s_avg,v_avg,dist,ang,circularity(v)]#hcolor,scolor,vcolor.v['center_x'],v['center_y']
        features.append(vector)
        #,v['size'],,aspect_ratio(v),h_min,h_max
        # imgvectors.append(vector)
    # vectors.append(imgvectors)
    i=i+1
    print(f'{int(i)}/{len(images)}')

  return features

def scale(data):
  features=scaler.fit_transform(data)*np.array([6,4,4,3,3,4])#best:[6,5,4,4,3,3,4]  69.74%
  vectors=features.reshape(-1,4,6)
  return features,vectors



In [ ]:
# a = np.arange(6).reshape((-1, 2))
# a

In [ ]:
import random
def getRandomSample(count):
  random.seed(44)
  indexes = random.sample(range(len(images)), count)
  random_images = [images[i] for i in indexes]
  random_labels = [true_labels[i] for i in indexes]
  return random_images,random_labels


In [ ]:
from sklearn.cluster import DBSCAN
# my_array,label_array= getRandomSample(len(images));
# feature_list=findFeatures(my_array)
feat,vect=scale(feature_list)
cluster = KMeans(n_clusters=40, n_init=10, random_state=42).fit(np.array(feat))
labelsDB = cluster.labels_


In [ ]:
# # wcss = []
# # for n_clusters in range(1,100):
# #     kmeans = KMeans(n_clusters=n_clusters, n_init=10)
# #     kmeans.fit(np.array(feat))
# #     wcss.append(kmeans.inertia_)
# #     # print(wcss)


# # plt.plot(range(1,100), wcss)

# plt.title('Elbow Method')
# plt.xlabel('Number of Clusters')
# plt.ylabel('WCSS')
# plt.show()

In [ ]:
def groupFeaturesByLabels(data,labels):
  group={}
  pou={}
  for vec,label in zip(data,labels):
    if(label not in group.keys()):
      group[label]=[]
    group[label].append(vec)
    pou[f'{vec}']=label

  return group,pou
gro,pou=groupFeaturesByLabels(feat,labelsDB)

In [ ]:
def featureCount(labelsDB,vectors,pou):
  count=1+np.amax(labelsDB)
  imgFeat=[]
  for image_vectors in vectors:
    feat=np.zeros(count)
    for vec in image_vectors:
      g=pou[f'{vec}']
      feat[g]=feat[g]+1
    imgFeat.append(feat)
  return np.array(imgFeat)

featurecount=featureCount(labelsDB,vect,pou)
featurecount

In [ ]:
####################################
classify(featurecount,label_array )
####################################

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score,f1_score,ConfusionMatrixDisplay

def classifyAndshowMetrics(datapoints, labels):
  test_size = 0.2
  X_train, X_test, y_train, y_test = train_test_split(datapoints, labels, test_size=test_size, random_state=42)

  clf = RandomForestClassifier(n_estimators=100, random_state=42)

  clf.fit(X_train, y_train)

  y_pred = clf.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='macro')
  recall =recall_score(y_test, y_pred, average='macro')
  f1 =f1_score(y_test, y_pred, average='macro')

  print("Precision:", precision)
  print("Recall:", recall)
  print("F1-score:", f1)
  print(f"Accuracy: {accuracy * 100:.2f}%")

  cm=confusion_matrix(y_test, y_pred)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm)
  disp.plot()
  plt.show()

In [ ]:
classifyAndshowMetrics(featurecount,label_array  )#true_labels

In [ ]:
import seaborn as sns

correlation_matrix = np.corrcoef(feat, rowvar=False)
sns.heatmap(correlation_matrix, annot=True)

print(correlation_matrix)

In [ ]:
# data = create(convertToHSV(images[123]))
# kmeans = KMeans(n_clusters=4, n_init=10).fit(data['info'])
# labels = kmeans.labels_
# plt.subplot(2, 2, 1)
# plt.imshow(np.array(images[123]).reshape((499, 499,3)))
# plt.axis('off')
# plt.subplot(2, 2, 2)
# plt.scatter(data['x'], data['y'], c=labels, cmap='viridis')
# plt.axis('off')
# plt.show()


# Plot results

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score,f1_score,ConfusionMatrixDisplay

def classifyAndshow(datapoints, labels):
  test_size = 0.2
  X_train, X_test, y_train, y_test = train_test_split(datapoints, labels, test_size=test_size, random_state=42)

  clf = RandomForestClassifier(n_estimators=100, random_state=42)

  clf.fit(X_train, y_train)

  y_pred = clf.predict(X_test)
  s=[]
  for a in X_test:
    i=0;
    for d in datapoints:
      if(np.array_equal(a,d)):
         s.append(i)
         break
      i=i+1
  imgs=[my_array[i] for i in s]
  labl=[label_array[i] for i in s]
  m=0;
  for i,j,z in zip(imgs,labl,y_pred):
      plt.figure(m)
      plt.imshow(np.array(i).reshape((499, 499,3)))
      plt.title(f'label:{j},pred:{z}')
      m=m+1

  plt.show()


In [ ]:
classifyAndshow(featurecount,label_array)